In [391]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt

'''g - growth 
s - switching
b - branching
r - retracting
d - dieing
n - new'''

'''
g + b + s = 1

'''
final_csv = ''
g_d_rate = 10
angle = 0.523599
population = ['g', 's', 'b']
time_to_eat_food = 1

def find_childless(nodes):
    childless_nodes_key = []
    for i in nodes:
        if len(nodes[i]['children']) == 0:
            childless_nodes_key.append(i)
    return childless_nodes_key

def find_node_parent(node_id, nodes):
    for i in nodes:
        if node_id in nodes[i]['children']:
            return nodes[i]

def check_node_near_food(node, food):
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) < food[i]['r']:
            food[i]['count'] += 1
            type_ret = food[i]['type']
            if (food[i]['count'] >= time_to_eat_food):
                del food[i]
            return [True, type_ret, food]
    return [False, False, food]

def delete_children_nodes(node_id, nodes):
    for i in nodes:
        if i in nodes[i]['children']:
            nodes[i]['children'].remove(i)
    return nodes


def find_closest_food(node, food):
    max_val = float('inf')
    closest_food = {'cord':np.array([0,0,0]), 'r':1, 'type':True, 'count': 0}
    for i in range(len(food)):
        if np.linalg.norm(node['cord'] - np.array(food[i]['cord'])) < max_val:
            closest_food = food[i]
    print(closest_food)
    return closest_food

def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def create_new_node(prent_node_id, nodes, food):
    closest_food = find_closest_food(nodes[prent_node_id], food)
    axis = nodes[prent_node_id]['cord']
    v = -axis + closest_food['cord']
    v = (v/np.linalg.norm(v))*g_d_rate
    v_1 = np.dot(rotation_matrix(axis, angle), v)
    v_2 = np.dot(rotation_matrix(axis, -angle), v)
    max_key = max(list(nodes.keys()))
    nodes[max_key+1] = {
        'cord': axis + g_d_rate*np.array([random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]),
        'children': [],
        'state': 'g'
    }
    nodes[max_key+2] = {
        'cord': axis + g_d_rate*np.array([random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]),
        'children': [],
        'state': 'g'
    }
    nodes[prent_node_id]['children'].extend([max_key+1, max_key+2])
    return nodes

def plot_values(nodes, t, final_csv):
    for key in nodes:
        final_csv += str(nodes[key]['cord'][0]) + ',' + str(nodes[key]['cord'][1]) + ',' + str(nodes[key]['cord'][2]) + ',' + str(t) + '\n'
    return final_csv

def get_csv(t, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu, food):
    final_csv= ''
    nodes = {}
    nodes[0] = {
        'cord': np.array([1, 4, 0]),
        'children': [],
        'state': 'g'
    }
    for j in range(t):
        print(j)
        childless_nodes_key = find_childless(nodes)
        for k, i in enumerate(childless_nodes_key):
            if nodes[i]['state'] == 'r':
                parent = find_node_parent(i, nodes)['cord']
                if np.linalg.norm(parent - nodes[i]['cord']) < g_d_rate:
                    del nodes[i]
                    nodes = delete_children_nodes(i, nodes)
                else:
                    nodes[i]['cord'] = nodes[i]['cord'] - g_d_rate
            else:
                is_near_food = check_node_near_food(nodes[i], food)
                food = is_near_food[2]
                if is_near_food[0] and is_near_food[1]:
                    weights = p_pos
                elif is_near_food[0] and (is_near_food[1] == False):
                    weights = p_neg
                else:
                    weights = p_neu
                new_state = random.choices(population, weights=weights, k=1)[0]
                if new_state == 's':
                    nodes[i]['state'] = 'r'
                else:
                    nodes[i]['state'] = new_state
                    if new_state == 'g':
                        parent = find_node_parent(i, nodes)['cord']
                        nodes[i]['cord'] = parent + (-parent + ((nodes[i]['cord'] + g_d_rate)))
                    else:
                        nodes = create_new_node(i, nodes, food)
        final_csv = plot_values(nodes, j, final_csv)
    #print(nodes)
    text_file = open("res.txt", "w")
    n = text_file.write(final_csv)
    text_file.close()

In [432]:
t = 60
#['g', 's', 'b']
p_neg = [0.7, 0.2, 0.1]
p_pos = [0.7, 0.1, 0.2]
p_neu = [0.7, 0.1, 0.2]
b_pos = 1
b_neg = 1
b_neu = 1
food = [{'cord':np.array([10,1,20]), 'r':10, 'type':True, 'count': 0}, 
        {'cord':np.array([9,7,5]), 'r': 10, 'type':False, 'count': 0},
        {'cord':np.array([6,8,8]), 'r': 10, 'type':False, 'count': 0},
        {'cord':np.array([2,3,3]), 'r': 10, 'type':False, 'count': 0},
        {'cord':np.array([3,5,0]), 'r': 10, 'type':True, 'count': 0},
        {'cord':np.array([1,4,2]), 'r':10, 'type':False, 'count': 0}]
get_csv(t, p_neg, p_pos, p_neu, b_pos, b_neg, b_neu, food)

0
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
1
2
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
3
4
5
6
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
7
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
8
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
9
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
10
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
11
12
13
14
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
15
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
16
17
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': ar

{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
50
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, '

57
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, '

{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}
{'cord': array([1, 4, 2]), 'r': 10, 'type': False, 'count': 0}


In [11]:
v = [3, 5, 0]
axis = [4, 4, 1]
theta = 1.2 

print(np.dot(rotation_matrix(axis, theta), v)) 
# [ 2.74911638  4.77180932  1.91629719]

[2.74911638 4.77180932 1.91629719]


In [36]:
nodes

{1: {'cord': [1, 2, 3], 'children': [], 'state': 'n'}}

In [40]:
max(list(nodes.keys()))

1